# Tutorial

## Setting up

For this tutorial, we use a real dataset.
The following bash script fetches the [Tycho dataset (v1.0)](https://www.tycho.pitt.edu/)
and builds an SQLite database from it. If the SQL database has already been built, the script will fail with
a harmless error message.

That database is then used to illustrate how to use `tablecloth` in this tutorial.


In [1]:
%%script bash
git clone https://github.com/lgautier/project-tycho-utilities.git 
cd project-tycho-utilities/
DBNAME=../tycho.db make all

make: Nothing to be done for 'all'.


fatal: destination path 'project-tycho-utilities' already exists and is not an empty directory.


Once the previous cell has completed its execution, we open a database connection and will use it
to evaluate SQL queries.

In [2]:
import sqlite3
dbfilename = "./tycho.db"
dbcon = sqlite3.connect(dbfilename)
cursor = dbcon.cursor()

## Using a query namespace (queryspace)

The package `tablecloth` is offering an alternative to ORMs where the handling of SQL code is preferred
(for example for legacy, or portability reasons).

A query on our dataset could be a selection of locations of interest. The SQL query to select rows
in the table `location` could then be like follows.

In [3]:
sql_locations = """
SELECT state, city
FROM location
WHERE
  state LIKE 'M%'
AND
  city IS NOT NULL
"""
cursor.execute(sql_locations)

Whenever exploring the content of a database, it is often desirable to quickly check a given
query returns the expected results through summary statistics.

Assuming that counting the number of cities for each selected state (here US states starting with the letter `M`)
is that summary statistics, this could be done in Python as shown in the cell right below, but it is generally better to push
that operation to the DBM.

In [4]:
from collections import Counter
ct = Counter(state for (state, city) in cursor)
print(ct.most_common(n=5))

[('MA', 66), ('MI', 25), ('MN', 9), ('ME', 8), ('MO', 7)]


If SQL should be written, one will often resort to using string formatting to reuse the previous query code.

In [5]:
sql_count = """
SELECT
  state,
  COUNT(city) AS ct
FROM
  {select_locations}
GROUP BY
  state
ORDER BY
  ct DESC
LIMIT 5
"""
cursor.execute(sql_count.format(select_locations='(%s)' % sql_locations))
cursor.fetchall()

[('MA', 66), ('MI', 25), ('MN', 9), ('ME', 8), ('MO', 7)]

The package `tablecloth` uses a "query namespace" ("queryspace" in short) to
help with managing the construction of such queries.

First, we can create a "queryspace".

In [6]:
from tablecloth import queryspace

space = queryspace.QuerySpace()

We can then store our first query in the queryspace.

In [7]:
space['select_locations'] = queryspace.QueryTemplate(
    'select_locations',
     sql_locations)

Now whenever we need the SQL code we can simply ask our queryspace to compile that SQL code.

In [8]:
print(space.compile('select_locations', {}))


SELECT state, city
FROM location
WHERE
  state LIKE 'M%'
AND
  city IS NOT NULL



This is arguably pointless when we have one query in the query namespace,
but the `tablecloth` will free the user from handling dependencies that can be
specified through string formatting.

We can add our second SQL query, where we compute counts, to the namespace.

In [9]:
sql_count = """
SELECT
  state,
  COUNT(city) AS ct
FROM
  {{select_locations}}
GROUP BY
  state
ORDER BY
  ct DESC
LIMIT 5
"""
space['count_cities'] = queryspace.QueryTemplate(
    'count_cities',
    sql_count)

When asking the queryspace to compile the SQL for that second query,
we call `count_cities` in the queryspace, we will obtain SQL that includes dependencies.

In [10]:
print(space.compile('count_cities', {}))

WITH select_locations AS (

SELECT state, city
FROM location
WHERE
  state LIKE 'M%'
AND
  city IS NOT NULL
)


SELECT
  state,
  COUNT(city) AS ct
FROM
  select_locations
GROUP BY
  state
ORDER BY
  ct DESC
LIMIT 5



The query returns the expected results.

In [11]:
cursor.execute(space.compile('count_cities', {}))
print(cursor.fetchall())

[('MA', 66), ('MI', 25), ('MN', 9), ('ME', 8), ('MO', 7)]


We can change the SQL for queries, and whenever these are dependencies
for other queries updated SQL will be produced.

In [12]:
space['select_locations'] = queryspace.QueryTemplate(
    'select_locations',
    """
    SELECT state, city
    FROM location
    WHERE state LIKE 'A%'
    """)
cursor.execute(space.compile('count_cities', {}))
print(cursor.fetchall())

[('AL', 5), ('AR', 4), ('AZ', 2), ('AK', 0), ('AS', 0)]


An arbitrary number of dependencies of SQL subqueries can be handled.

In [13]:
space['select_locations_with_count'] = queryspace.QueryTemplate(
    'select_locations_with_count',
    """
    SELECT state, city, ct
    FROM
    {{select_locations}}
    INNER JOIN
    {{count_cities}}
    USING
    (states)
    """)
print(space.compile('select_locations_with_count', {}))

WITH select_locations AS (

    SELECT state, city
    FROM location
    WHERE state LIKE 'A%'
    ),
count_cities AS (

SELECT
  state,
  COUNT(city) AS ct
FROM
  select_locations
GROUP BY
  state
ORDER BY
  ct DESC
LIMIT 5
)


    SELECT state, city, ct
    FROM
    select_locations
    INNER JOIN
    count_cities
    USING
    (states)
    


## Queryspaces can help manage collections of interdependent subqueries

**NOTE: this is very much WIP. There is more to come.**

The labels (query names) in a query space are easily accessible.

In [14]:
space.available_nodes

{'count_cities', 'select_locations', 'select_locations_with_count'}

The queryspace can also indicate whether there is a transitive dependency
(direct or indirect dependency) between subqueries.

In [15]:
space.find_in_dependencies('select_locations_with_count', 'count_cities')

True